In [50]:
import sys
sys.path.append('../cfq')

from itertools import *
import pickle
import numpy as np
from data import RaggedArray, CFQDataset

np.set_printoptions(linewidth=150)

In [51]:
input_dir = '/work/yu_gai/cfq/data/cfq'

In [52]:
df = sqlCtx.read.parquet(f'/data/yu_gai/cfq/dataset.parquet/').sort('index').persist()
question = df.rdd.map(lambda r: r['questionPatternModEntities']).collect()
template = df.rdd.map(lambda r: r['questionTemplate']).collect()
sparql = df.rdd.map(lambda r: r['sparqlPatternModEntities']).collect()

In [53]:
dat = np.load(f'{input_dir}/data.npz')
idx = np.load(f'{input_dir}/splits/mcd1.npz')['testIdxs']
tok_vocab = idx2tok, _ = pickle.load(open(f'{input_dir}/tok-vocab.pickle', 'rb'))
tag_vocab = idx2tag, _ = pickle.load(open(f'{input_dir}/tag-vocab.pickle', 'rb'))
typ_vocab = idx2typ, _ = pickle.load(open(f'{input_dir}/typ-vocab.pickle', 'rb'))
dataset = CFQDataset(idx, dat, tok_vocab, tag_vocab, typ_vocab)
getitem = lambda key: dataset[np.where(idx == key)[0].item()]

KeyError: 'seq_tag is not a file in the archive'

In [29]:
out = np.load(f'/data/yu_gai/cfq/None_010621_081316/test_dict.npz')

In [30]:
n = np.array([getitem(index)['n'] for index in out['index']])
m = n * n
indptr = np.hstack([np.zeros(1), m.cumsum()]).astype(int)

In [31]:
def pos_typ(mem, n_mem, src, dst, typ):
    mem = RaggedArray(mem, np.cumsum(np.hstack([[0], n_mem])))
    i, j = np.where(typ)
    rels = []
    for u, t, v in zip(src[i], j, dst[i]):
        for x, y in product(mem[u], mem[v]):
            rels.append(f'{idx2tok[y]} {idx2typ[t]} {idx2tok[x]}')
    print(*sorted(rels), sep='\n')

In [32]:
out['em'].mean()

0.8941228079337393

In [49]:
idx2tag.extend(['NP', 'vars'])
for em, index, start, end in islice(zip(out['em'], out['index'], indptr[:-1], indptr[1:]), 100):
    if not em:
        dat = getitem(index)
        
        print(question[index])
        print(' '.join(idx2tag[idx] for idx in dat['seq']))
        print(' '.join(idx2tag[idx] for idx in dat['seq_tag']))
        for seq_noun in dat['seq_noun']:
            print(' '.join(idx2tag[idx] for idx in seq_noun))
            
        print('*' * 75)
        
        isrel = lambda r: all(s not in r for s in ['!=', ' a ', 'ns:people.person.gender', 'ns:people.person.nationality'])
        print(*sorted(filter(isrel, sparql[index].split('\n')[1 : -1])), sep='\n')
#         print(dat['pos_noun'], dat['pos_tag'], dat['pos_np'], dat['pos_nps'])
        print(dat['isnp'], dat['pos_nps'], dat['pos_tag'])
        
        templ = ' '.join([idx2tag[idx] for idx in dat['seq']])
        src, dst = dat['src'], dat['dst']
        typ_true = out['typ_true'][start : end]
        typ_pred = out['typ_pred'][start : end]
#         print(src, dst)
#         print(typ_true.astype(int))
#         print(typ_pred.astype(int))
        
        print('*' * 75)
        pos_typ(dat['mem'], dat['n_mem'], src, dst, typ_true)
        
        print('*' * 75)
        pos_typ(dat['mem'], dat['n_mem'], src, dst, typ_pred)
        
        print('*' * 75)

What was written by a Spanish art director , written by M3 's writer , and executive produced by M0
What was [VP_SIMPLE] by a [ADJECTIVE_SIMPLE] [NP_SIMPLE] , [VP_SIMPLE] by [entity] 's [ROLE_SIMPLE] , and [VP_SIMPLE] by [entity] {SEP} ?x0 ?x1 ?x2 {NIL}
NP was [VP_SIMPLE] by NP , [VP_SIMPLE] by NP , and [VP_SIMPLE] by NP {SEP} vars {NIL}
What
a [ADJECTIVE_SIMPLE] [NP_SIMPLE]
[entity] 's [ROLE_SIMPLE]
[entity]
?x0 ?x1 ?x2
***************************************************************************
?x0 ns:film.film.executive_produced_by M0 .
?x0 ns:film.film.written_by ?x1 .
?x0 ns:film.film.written_by ?x2 .
?x2 ns:film.writer.film M3
[ True False False False  True  True  True False False False  True  True  True False False False False  True False  True  True  True False] [ 0  0  0  0  2  3  4  0  0  0  6  7  8  0  0  0  0 10  0 12 13 14  0] [ 0  1  2  3  0  0  0  5  6  7  0  0  0  9 10 11 12  0 14  0  0  0 16]
***************************************************************************
?x

***************************************************************************
?x0 ns:influence.influence_node.influenced ?x1
?x0 ns:influence.influence_node.influenced ?x2
?x1 ns:film.actor.film/ns:film.performance.film M1
?x2 ns:film.actor.film/ns:film.performance.film ?x3
?x2 ns:film.cinematographer.film ?x3
?x2 ns:film.writer.film ?x3
?x3 ns:film.film.prequel M2
***************************************************************************
?x0 ns:influence.influence_node.influenced ?x1
?x0 ns:influence.influence_node.influenced ?x2
?x1 ns:film.actor.film/ns:film.performance.film ?x3
?x2 ns:film.actor.film/ns:film.performance.film ?x3
?x2 ns:film.cinematographer.film ?x3
?x2 ns:film.writer.film ?x3
?x3 ns:film.film.prequel M2
***************************************************************************
Did M2 influence a film director 's child and influence a film editor
Did [entity] [VP_SIMPLE] a [NP_SIMPLE] 's [ROLE_SIMPLE] and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 ?x2 {NIL}
Did NP [VP_

In [254]:
[templ for templ in templ_train if templ.startswith("Did a [ADJECTIVE_SIMPLE]")]

[]

In [168]:
ts = {' '.join([idx2tag[idx] for idx in getitem(index)['seq']]) for em, index in zip(out['em'], out['index']) if not em}

In [144]:
idx_train

array([166986, 239216, 177275, ..., 183022, 185019, 139419])

In [217]:
idx_train = np.load(f'{input_dir}/splits/mcd3.npz')['trainIdxs']
ds_train = CFQDataset(idx_train, dat, tok_vocab, tag_vocab, typ_vocab)
get_train = lambda key: ds_train[np.where(idx_train == key)[0].item()]
templ_train = {' '.join([idx2tag[idx] for idx in get_train(index)['seq']]) for index in idx_train}

In [218]:
idx_test = np.load(f'{input_dir}/splits/mcd3.npz')['testIdxs']
ds_test = CFQDataset(idx_test, dat, tok_vocab, tag_vocab, typ_vocab)
get_test = lambda key: ds_test[np.where(idx_test == key)[0].item()]
templ_test = {' '.join([idx2tag[idx] for idx in get_test(index)['seq']]) for index in idx_test}

In [219]:
both = templ_train.intersection(templ_test)
print(len(templ_train), len(templ_test), len(both))
list(islice(both, 10))

9148 1058 210


["Who was a [NP_SIMPLE] 's [ROLE_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 'Which [ADJECTIVE_SIMPLE] [NP_SIMPLE] was [VP_SIMPLE] by [entity] and was [VP_SIMPLE] by [entity] {SEP} ?x0 {NIL}',
 'Which [ROLE_SIMPLE] of a [NP_SIMPLE] was [VP_SIMPLE] by [entity] {SEP} ?x0 ?x1 {NIL}',
 'What did [entity] [VP_SIMPLE] and [entity] [VP_SIMPLE] {SEP} ?x0 {NIL}',
 'Which [NP_SIMPLE] was [VP_SIMPLE] by and [VP_SIMPLE] by [entity] {SEP} ?x0 {NIL}',
 "Which [ROLE_SIMPLE] of [entity] was [entity] 's [ROLE_SIMPLE] {SEP} ?x0 {NIL}",
 "Which [NP_SIMPLE] was a [NP_SIMPLE] 's [ROLE_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 'What [ADJECTIVE_SIMPLE] [NP_SIMPLE] was a [ROLE_SIMPLE] of [entity] {SEP} ?x0 {NIL}',
 "What [ROLE_SIMPLE] of [entity] was [entity] 's [ROLE_SIMPLE] {SEP} ?x0 {NIL}",
 'Which [ROLE_SIMPLE] of [entity] [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}']

In [268]:
import re
from operator import *

In [271]:
def n_var(r):
    return len(set(re.findall(r'\?x\d', r['sparqlPatternModEntities'])))

def n(r):
    return sum(1 for tok in r['questionTemplate'].split(' ') if tok in ['[NP_SIMPLE]', '[ROLE_SIMPLE]', 'Who'])

# n_var(df.rdd.take(1)[0])
# n(df.rdd.take(1)[0])
df.rdd.map(lambda r: n_var(r) == n(r)).reduce(add)

115974

In [273]:
df.rdd.filter(lambda r: n_var(r) != n(r)).map(lambda r: r['questionTemplate']).take(10)

["Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity] , [entity] , and [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] , [VP_SIMPLE] , [VP_SIMPLE] , and [VP_SIMPLE] [entity]",
 "Did [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] , [ROLE_SIMPLE] , and [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [entity] and [VP_SIMPLE] a [NP_SIMPLE] 's [ROLE_SIMPLE]",
 "Did [entity] [VP_SIMPLE] [entity] 's [ROLE_SIMPLE] , [VP_SIMPLE] a [ROLE_SIMPLE] and [ROLE_SIMPLE] of [entity] and [entity] , and

In [ ]:
df.rdd.map(lambda r: r['questionTemplate']).filter(lambda r: r.startswith(''))

In [270]:
df.count()

239357

In [261]:
print(df.rdd.map(lambda r: r['sparqlPatternModEntities']).take(1)[0])

SELECT count(*) WHERE {
?x0 ns:film.actor.film/ns:film.performance.character M1 .
?x0 ns:film.editor.film M0 .
?x0 ns:film.producer.film|ns:film.production_company.films M0 .
?x0 ns:people.person.gender ns:m.02zsn
}


In [255]:
df.columns

['complexityMeasures',
 'expectedResponse',
 'expectedResponseWithMids',
 'index',
 'question',
 'questionPatternModEntities',
 'questionTemplate',
 'questionWithBrackets',
 'questionWithMids',
 'ruleIds',
 'ruleTree',
 'sparql',
 'sparqlPattern',
 'sparqlPatternModEntities']

In [220]:
sum(1 for index in idx_test if ' '.join([idx2tag[idx] for idx in get_test(index)['seq']]) in both)

3081

In [247]:
[x for x in templ_train if x.startswith("Was [entity] 's")]

["Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 ?x2 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Was [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 ?x2 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] 's [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Was [entity] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Was [entity] 's [

In [249]:
[x for x in templ_train if "a [NP_SIMPLE]" in x]

["Who was [VP_SIMPLE] by [entity] 's [ROLE_SIMPLE] and [VP_SIMPLE] by a [ROLE_SIMPLE] of a [NP_SIMPLE] {SEP} ?x0 ?x1 ?x2 ?x3 {NIL}",
 'What was [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , [VP_SIMPLE] by , and [VP_SIMPLE] by a [NP_SIMPLE] that [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}',
 'What was [VP_SIMPLE] by a [NP_SIMPLE] that [entity] was [VP_SIMPLE] by and [VP_SIMPLE] by and [VP_SIMPLE] by [entity] {SEP} ?x0 ?x1 {NIL}',
 'Which [NP_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] that [entity] was [VP_SIMPLE] by and [entity] [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}',
 'Did [entity] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] of [entity] , and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}',
 "Was [entity] a [NP_SIMPLE] 's [ROLE_SIMPLE] 's [ROLE_SIMPLE] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] {SEP} ?x0 ?x1 ?x2 {NIL}",
 'Who was [VP_SIMPLE] by a [NP_SIMPLE] and was [VP_SIMPLE] by a [ROLE_SIMPLE] of a [ROLE_SIMPLE] of [entity] {SEP} ?x0 ?x1 ?x2 ?x3 {NIL}',
 'What [NP_SIMP

In [241]:
templ_test.difference(templ_train)

{"Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] and [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] a [NP_SIMPLE] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] [entity] , and [VP_SIMPLE] [entity] {SEP} ?x0 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP_SIMPLE] [entity] , [VP_SIMPLE] a [NP_SIMPLE] , and [VP_SIMPLE] [entity] {SEP} ?x0 ?x1 {NIL}",
 "Did [entity] 's [ADJECTIVE_SIMPLE] [ROLE_SIMPLE] [VP

In [243]:
set.difference?